
#### Group Members:
Kinjal  
Tim  
Doris  
Zuqi  
Tania  

# I. Data Sources and Justificiation
The report is expected to begin with a description of the data sources used (at least two core data resources). The participants should describe:

1 Why the data is of interest (what is the purpose of this report?)  
2 Where the data was obtained  
3 Any associated methodologies that might be of interest for the report  
4 How do data resources relate to one another? 
  
This section is worth 30% of the total grade.  It will be marked on completeness, on the quality of the written product, on the suitability of the data resources used, the use of secondary data resources (such as the ISO national codes) and the complexity of the data resource.


***
# II. Data Model and Implementation
***

#### (a) Creating and connecting to the database:
The database was created through the GUI interface pgAdmin4. A json file was used to connect to the database in postgres. Edit the json file (username and password) to make a successful connection. 

In [2]:
#Importing all necessary packages and tools
import json        
import csv
import re
import psycopg2

#Opening the connection string stored in a json file
with open("/Users/Kinjal Majumdar/awsconnect.json") as f:
    data = json.load(f)
conn = psycopg2.connect(**data)
cur = conn.cursor()

#### (b) Creating the schema:

In [3]:
conn.rollback()
cur.execute("""CREATE SCHEMA auto AUTHORIZATION postgres""")
conn.commit()
#enter your postgres username where user_name is mentioned

#### (c) Creating the basic table structure
In this step, all temporary and permanent tables (except for the three connecting main tables: fuel_consumption, sales, car) are created. The temporary tables have the same structure as the data files that are used in the following step to populate the database. They help us work with the data inside the database and are necessary for reference when connecting the permanenet tables. The smaller tables (brand, model, type, transmission, drive, fuel, segment) are created to store data from the temporary files. 

In [4]:
conn.rollback() #undoing all changes from the current transaction

basicstructure = """

CREATE TABLE auto.brand(  brandid SERIAL             PRIMARY KEY,
                              make CHARACTER VARYING);
							  
CREATE TABLE auto.model(  modelid SERIAL             PRIMARY KEY,
                              model CHARACTER VARYING);
							  
CREATE TABLE auto.type(  typeid SERIAL         PRIMARY KEY,
                            model_type CHARACTER VARYING);
							
CREATE TABLE auto.transmission(  transid SERIAL         PRIMARY KEY,
                            transmission CHARACTER VARYING);	
							
CREATE TABLE auto.drive(  driveid SERIAL         PRIMARY KEY,
                            drive CHARACTER VARYING);
							
CREATE TABLE auto.fuel(  fuelid SERIAL         PRIMARY KEY,
                            fuel CHARACTER VARYING);
							
CREATE TABLE auto.segment(  segid SERIAL         PRIMARY KEY,
                            segment CHARACTER VARYING);							


CREATE TABLE auto.temporary_sales (
							make CHARACTER VARYING,
							model CHARACTER VARYING,
							model_type CHARACTER VARYING,
							sales CHARACTER VARYING,
							calendar_year INTEGER); /* CHANGED IT TO INTEGER */
							
CREATE TABLE auto.temporary_fuel_economy ( 
							make CHARACTER VARYING,
							model CHARACTER VARYING,
							model_type CHARACTER VARYING,
							transmission CHARACTER VARYING,
							drive CHARACTER VARYING,
							fuel CHARACTER VARYING,
							segment CHARACTER VARYING,
							fuel_economy INTEGER,  /* CHANGED IT TO INTEGER */
							calendar_year INTEGER
							);
                       """
cur.execute(basicstructure)
conn.commit() #Committing to all the data changes made in this particular query

#### (d) Populating temporary tables with data from csv files
In this step, the csv files are imported and the data is inserted into the temporary tables (temporary_fuel_economy, temporary_sales). The structure of the temporary tables matches that of the csv files. 

In [5]:
# Enter your file path into the variables 'file_path_fuel_economy' and 'file_path_Sales'. 
# Please ensure that you are using a forward slash to separate each folder

file_path_fuel_economy = '/Users/Kinjal Majumdar/Downloads/fuel_economy.csv'
file_path_Sales = '/Users/Kinjal Majumdar/Downloads/Sales.csv'

In [6]:
# Inserting data into temporary_fuel_economy #ADD VARIABLE FOR FILES
insertfueleconomy = '''INSERT INTO auto.temporary_fuel_economy VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

with open(file_path_fuel_economy,'r') as temp:
    inputdata = csv.reader(temp)
    next(inputdata)
    for rows in inputdata:
        cur.execute(insertfueleconomy,rows)
        conn.commit()

In [7]:
# Inserting data into temporary_sales
insertsales = '''INSERT INTO auto.temporary_sales VALUES(%s,%s,%s,%s,%s)'''

with open(file_path_Sales,'r') as temp:
    inputdata = csv.reader(temp)
    next(inputdata)
    for rows in inputdata:
        cur.execute(insertsales,rows)
        conn.commit()

#### (e) Populating permanent tables with data from temporary tables
The data from temporary tables is inserted into permanent tables. This step helps us normalize the data and create unique ID's for the tables make, model, type, drive, fuel, transmission and segment. The permanent tables contain all unique data from BOTH the temporary sales and the temporary fuel consumption tables. This is necessary, because there are a few cars that only appear in one of the tables.

In [8]:
conn.rollback()
cur.execute("""
            INSERT INTO auto.brand(make) 
            SELECT DISTINCT upper(make)
            FROM auto.temporary_sales;
            
            INSERT INTO auto.brand(make) 
            SELECT DISTINCT upper(make)
            FROM auto.temporary_fuel_economy  fe 
            WHERE upper(fe.make) NOT IN (SELECT make from auto.brand);
            
            INSERT INTO auto.model(model)
            SELECT DISTINCT upper(model)
            FROM auto.temporary_sales;
            
            INSERT INTO auto.model(model) 
            SELECT DISTINCT upper(model)
            FROM auto.temporary_fuel_economy  fe
            WHERE upper(fe.model) NOT IN (SELECT model from auto.model);
            
            INSERT INTO auto.type(model_type)
            SELECT DISTINCT upper(model_type)
            FROM auto.temporary_sales;
            
            INSERT INTO auto.type(model_type) 
            SELECT DISTINCT upper(model_type)
            FROM auto.temporary_fuel_economy  fe
            WHERE upper(fe.model_type) NOT IN (SELECT model_type from auto.type);
            
            INSERT INTO auto.drive(drive)
            SELECT DISTINCT upper(drive)
            FROM auto.temporary_fuel_economy;
            
            INSERT INTO auto.fuel(fuel)
            SELECT DISTINCT upper(fuel)
            FROM auto.temporary_fuel_economy;
            
            INSERT INTO auto.transmission(transmission)
            SELECT DISTINCT upper(transmission)
            FROM auto.temporary_fuel_economy;
            
            INSERT INTO auto.segment(segment)
            SELECT DISTINCT upper(segment)
            FROM auto.temporary_fuel_economy;
            """)
conn.commit()

#### (f) Additional data cleaning: Removing spaces
When inserting the data, we realized that there are additional spaces in the model table. These are removed by the following query:

In [9]:
conn.rollback()
cur.execute("""select distinct(
            replace(model, ' ', '')) as model
            from auto.model
            group by model""")
conn.commit()

#### (g) Create and populate the car table:
This table stores all of the unique cars, defined by its make, model, type, drive, transmission, fuel and segment. However, instead of storing the actual make, model etc. each column contains an ID that is linked to the smaller tables that were created in the last step.  

In [10]:
conn.rollback()
cur.execute("""

CREATE TABLE auto.car(  car_ID SERIAL             PRIMARY KEY,
                            brand_ID 				INTEGER REFERENCES auto.brand, 
					        model_ID 				INTEGER REFERENCES auto.model,
                            type_ID 				INTEGER REFERENCES auto.type,
                            transmission_ID 		INTEGER REFERENCES auto.transmission,
                            drive_ID 				INTEGER REFERENCES auto.drive,
                            fuel_ID 				INTEGER REFERENCES auto.fuel,
					        segment_ID 				INTEGER REFERENCES auto.segment
                            
                           );

INSERT INTO auto.car (brand_ID, model_ID, type_ID, transmission_ID, drive_ID,fuel_ID, segment_ID)

select distinct master_table.brandid as brand_ID, 
master_table.modelid as model_ID, 
master_table.typeid as type_ID, 
master_table.transid as transmission_ID, 
master_table.driveid as drive_ID,
master_table.fuelid as fuel_ID,
master_table.segid as segment_ID
from (auto.temporary_fuel_economy
	  

full join auto.model
on upper(temporary_fuel_economy.model) = upper(model.model)

full join auto.brand
on upper(temporary_fuel_economy.make) = upper(brand.make)

full join auto.type
on upper(temporary_fuel_economy.model_type) = upper(type.model_type)

full join auto.transmission
on upper(temporary_fuel_economy.transmission) = upper(transmission.transmission)

full join auto.drive
on upper(temporary_fuel_economy.drive) = upper(drive.drive)

full join auto.fuel
on upper(temporary_fuel_economy.fuel) = upper(fuel.fuel)

full join auto.segment
on upper(temporary_fuel_economy.segment)= upper(segment.segment)) as master_table
												
order by brand_id ASC""")
conn.commit()

#### (h) Creating a temporary master table for joining the car table with fuel_consumption and sales 
This table helps us connect the car table, which only includes ID's and not the actual data that describes each car, with the fuel consumption and the sales table. This temporary table is not necessary to populate the other tables but it serves as a reference and helps us check if the data is inserted and connected correctly in the database. 

In [11]:
conn.rollback()
cur.execute("""
CREATE TABLE auto.temp_master(
							car_ID						INTEGER						PRIMARY KEY,
                            brand_ID 					INTEGER						, 
					  		brand 						CHARACTER VARYING			,
					   		model_ID 					INTEGER						,
							model 						CHARACTER VARYING			,
							modeltype_ID 				INTEGER						,
							modeltype 					CHARACTER VARYING			,
							transmission_ID 			INTEGER						,
							transmission 				CHARACTER VARYING			,
							drive_ID 					INTEGER						,
							drive 						CHARACTER VARYING			,
							fuel_ID 					INTEGER						,
							fuel 						CHARACTER VARYING			,
							segment_ID 					INTEGER						,
							segment 					CHARACTER VARYING			
							 );

insert into auto.temp_master 

select ca.car_id, ca.brand_id, br.make as brand, ca.model_id, mo.model, ca.type_id, ty.model_type, 
ca.transmission_id, tr.transmission, ca.drive_id, dr.drive, ca.fuel_id, fu.fuel, ca.segment_id, se.segment 
from auto.car ca

join auto.brand br
on br.brandid =ca.brand_id

join auto.model mo
on mo.modelid = ca.model_id

join auto.type ty
on ty.typeid = ca.type_id

join auto.transmission tr
on tr.transid = ca.transmission_id

join auto.drive dr
on dr.driveid = ca.drive_id

join auto.fuel fu
on fu.fuelid = ca.fuel_id

join auto.segment se
on se.segid = ca.segment_id""")

conn.commit()

#### (i) Creating the fuel_consumption table
In this step, the fuel_consumption table is created. It stores the fuel economy rating of each vehicle for each year. It is linked to a specific car through the car_ID. The temp_master table is used as a reference to insert the correct values for each car_ID. 

In [12]:
conn.rollback()
cur.execute("""CREATE TABLE auto.fuel_consumption(fcid SERIAL             PRIMARY KEY,
                       car_ID 					INTEGER REFERENCES auto.car, 
                       calendar_year 			INTEGER,
					   fuel_economy 			INTEGER);
					   
insert into auto.fuel_consumption (car_ID, fuel_economy, calendar_year)

select temp_master.car_ID, tfe.fuel_economy as f_econ, tfe.calendar_year as cal_yr 
from auto.temporary_fuel_economy as tfe

join auto.temp_master 
on
upper(tfe.make) = upper(temp_master.brand)

AND upper(tfe.model) = upper(temp_master.model)
AND upper(tfe.model_type) = upper(temp_master.modeltype)
AND upper(tfe.transmission) = upper(temp_master.transmission)
AND upper(tfe.drive) = upper(temp_master.drive)
AND upper(tfe.fuel) = upper(temp_master.fuel)
AND upper(tfe.segment) = upper(temp_master.segment)
""")
conn.commit()

#### (j) Creating the sales table
In this step, the sales table is created. It stores the total sales number of each vehicle for each year. It is linked to a specific car through the car_ID. The temp_master table is used as a reference to insert the correct values for each car_ID. Due to the fact that our sales data only specified the make, model and type (not the different transmission, fuel options etc.), the sales table is only joined on those columns. 

In [13]:
conn.rollback()

cur.execute("""CREATE TABLE auto.sales (
                            sale_id SERIAL PRIMARY KEY,
                            car_ID INTEGER REFERENCES auto.car,
                            calendar_year INTEGER,
                            total_sales CHARACTER VARYING);
                            
insert into auto.sales (car_ID, calendar_year, total_sales)
select ma.car_id, sa.calendar_year, sa.sales
from auto.temp_master ma

join auto.temporary_sales sa
on upper(ma.brand) = upper(sa.make)
AND upper(ma.model) = upper(sa.model)
AND upper(ma.modeltype) = upper(sa.model_type)

""")
conn.commit()

#### (k) Creating and populating the fuel table:
This table stores the yearly average of fuel prices from 2013 to 2017

In [26]:
# Enter your file path into the variable 'file_path_fuel_price'.
# Please ensure that you are using a forward slash to separate each folder

file_path_fuel_price = '/Users/Kinjal Majumdar/Downloads/fuel_price.csv'


In [27]:
conn.rollback()
cur.execute("""CREATE TABLE auto.fuel_price (priceid SERIAL             PRIMARY KEY,
			    calendar_year INTEGER,
					    price DECIMAL);""")
conn.commit()

In [28]:
conn.rollback()
insertfuelprice = '''INSERT INTO auto.fuel_price VALUES(%s,%s,%s)'''

with open(file_path_fuel_price,'r') as temp:
    inputdata = csv.reader(temp)
    for rows in inputdata:
        cur.execute(insertfuelprice,rows)
        conn.commit()

#### (l) Changes to the database
The data in the sales table for total_sales is converted to INTEGER (from CHARACTER VARYING)

In [29]:
conn.rollback()
cur.execute("""
    update auto.sales set total_sales = replace(total_sales,',','') where total_sales like '%,%';
    alter table auto.sales alter column total_sales type INTEGER using total_sales::numeric;
    """)
conn.commit()

#### (m) Dropping all temporary tables:

In [244]:
conn.rollback()
cur.execute("""
    DROP TABLE auto.temporary_fuel_economy CASCADE;
    DROP TABLE auto.temporary_sales CASCADE;
    DROP TABLE auto.temp_master CASCADE;
    """)
conn.commit()

### X. Challenges

***
# III. Analysis 
***

### 1. OVERALL RANKING sales vs. fuel economy

### Segment 1 - Pickup

In [276]:
conn.rollback()

pickup = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 1 /* segment_ID 1 = pickup */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(pickup)
print_pickup = cur.fetchall()
print(print_pickup)

[('DIESEL', 'CHEVROLET', 'COLORADO', 'N/A', 24, 78253, 2017), ('GASOLINE', 'HONDA', 'RIDGELINE', 'N/A', 22, 34034, 2017), ('GASOLINE', 'TOYOTA', 'TACOMA', 'N/A', 20, 152226, 2017), ('DIESEL', 'GMC', 'CANYON', 'N/A', 24, 26345, 2017), ('GASOLINE', 'CHEVROLET', 'COLORADO', 'N/A', 21, 78253, 2017), ('GASOLINE', 'NISSAN', 'FRONTIER', 'N/A', 19, 54481, 2017), ('GASOLINE', 'GMC', 'CANYON', 'N/A', 21, 26345, 2017), ('GASOLINE', 'NISSAN', 'TITAN', 'N/A', 18, 39533, 2017), ('GASOLINE', 'TOYOTA', 'TUNDRA', 'N/A', 16, 81270, 2017)]


### Segment 2 - Station Wagon

In [266]:
conn.rollback()

stationwagon = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 2 /* segment_ID 2 = Station Wagon */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(stationwagon)
print_stationwagon = cur.fetchall()
print(print_stationwagon)

[('NISSAN', 'MURANO', 'N/A', 24, 60372, 2017), ('HONDA', 'FIT', 'N/A', 34, 41657, 2017), ('VOLVO', 'V60', 'N/A', 28, 3493, 2017), ('SUBARU', 'IMPREZA', 'N/A', 29, 91959, 2017), ('HONDA', 'HR-V', 'N/A', 30, 68936, 2017), ('CHEVROLET', 'BOLT', 'N/A', 119, 10822, 2017), ('INFINITI', 'QX50', 'N/A', 20, 15113, 2017), ('SUBARU', 'IMPREZA', 'N/A', 29, 65044, 2017), ('FIAT', '500L', 'N/A', 25, 994, 2017), ('VOLVO', 'V60', 'N/A', 28, 1964, 2017), ('NISSAN', 'JUKE', 'N/A', 29, 14308, 2017), ('KIA', 'SOUL', 'N/A', 27, 83720, 2017), ('KIA', 'NIRO', 'N/A', 49, 17112, 2017)]


### Segment 3 - Special Purpose

In [267]:
conn.rollback()

specialpurpose = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 3 /* segment_ID 3 = special purpose */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(specialpurpose)
print_specialpurpose = cur.fetchall()
print(print_specialpurpose)

[('NISSAN', 'NV200', 'N/A', 25, 15733, 2017), ('CHEVROLET', 'COLORADO', 'N/A', 15, 78253, 2017), ('GMC', 'CANYON', 'N/A', 15, 26345, 2017), ('MERCEDES-BENZ', 'METRIS', 'N/A', 22, 4347, 2017)]


### Segment 4 - Van

In [268]:
conn.rollback()

van = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 4 /* segment_ID 4 = van */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(van)
print_van = cur.fetchall()
print(print_van)

[]


### Segment 5 - Large Car

In [ ]:
conn.rollback()

largecar = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 5 /* segment_ID 5 = Large car */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(largecar)
print_largercar = cur.fetchall()
print(print_largecar)

### Segment 6 - Minivan

In [270]:
conn.rollback()

minivan = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 6 /* segment_ID 6 = minivan */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(minivan)
print_minivan = cur.fetchall()
print(print_minivan)

[('CHRYSLER', 'PACIFICA', 'N/A', 22, 87682, 2017), ('NISSAN', 'QUEST', 'N/A', 22, 13545, 2017), ('TOYOTA', 'SIENNA', 'N/A', 21, 91070, 2017), ('KIA', 'SEDONA', 'N/A', 20, 22463, 2017), ('HONDA', 'ODYSSEY', 'N/A', 22, 73595, 2017)]


### Segment 7 - Midsize Car

In [280]:
conn.rollback()

midsize = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 7 /* segment_ID 1 = pickup */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(midsize)
print_midsize = cur.fetchall()
print(print_midsize)

[('GASOLINE', 'MAZDA', '3', 'N/A', 30, 72121, 2017), ('ETHANOL/GAS', 'CHRYSLER', '200', 'N/A', 24, 23170, 2017), ('GASOLINE', 'VOLKSWAGEN', 'PASSAT', 'N/A', 25, 63935, 2017), ('GASOLINE', 'INFINITI', 'Q50', 'N/A', 24, 36575, 2017), ('GASOLINE', 'NISSAN', 'SENTRA', 'N/A', 30, 179949, 2017), ('GASOLINE', 'FORD', 'FUSION', 'N/A', 24, 177751, 2017), ('GASOLINE', 'NISSAN', 'ALTIMA', 'N/A', 29, 258948, 2017), ('GASOLINE', 'SUBARU', 'IMPREZA', 'N/A', 30, 91959, 2017), ('GASOLINE', 'TOYOTA', 'COROLLA', 'N/A', 31, 285023, 2017), ('GASOLINE', 'KIA', 'FORTE', 'N/A', 29, 87717, 2017), ('GASOLINE', 'MASERATI', 'GHIBLI', 'N/A', 19, 4953, 2017), ('GASOLINE', 'TOYOTA', 'AVALON', 'N/A', 24, 30086, 2017), ('GASOLINE', 'AUDI', 'A6', 'N/A', 26, 12873, 2017), ('GASOLINE', 'TOYOTA', 'PRIUS', 'N/A', 52, 70403, 2017), ('GASOLINE', 'BUICK', 'REGAL', 'N/A', 23, 11578, 2017), ('GASOLINE', 'NISSAN', 'MAXIMA', 'N/A', 25, 54590, 2017), ('GASOLINE', 'MAZDA', '6', 'N/A', 29, 37422, 2017), ('GASOLINE', 'JAGUAR', 'XF',

### Segment 8 - Small SUV

In [281]:
conn.rollback()

smallSUV = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 8 /* segment_ID 8 = Small SUV */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(smallSUV)
print_smallSUV = cur.fetchall()
print(print_smallSUV)

[('GASOLINE', 'JEEP', 'RENEGADE', 'N/A', 25, 88708, 2017), ('ETHANOL/GAS', 'GMC', 'TERRAIN', 'N/A', 25, 79993, 2017), ('ETHANOL/GAS', 'DODGE', 'JOURNEY', 'N/A', 19, 120400, 2017), ('GASOLINE', 'JEEP', 'WRANGLER', 'N/A', 18, 144589, 2017), ('GASOLINE', 'MAZDA', 'CX-9', 'N/A', 24, 22043, 2017), ('GASOLINE', 'SUBARU', 'OUTBACK', 'N/A', 25, 149002, 2017), ('DIESEL', 'JAGUAR', 'F-PACE', 'N/A', 29, 18344, 2017), ('ETHANOL/GAS', 'JEEP', 'CHEROKEE', 'N/A', 24, 142643, 2017), ('GASOLINE', 'JEEP', 'COMPASS', 'N/A', 24, 29932, 2017), ('GASOLINE', 'HONDA', 'PILOT', 'N/A', 22, 88284, 2017), ('ETHANOL/GAS', 'AUDI', 'Q5', 'N/A', 22, 39592, 2017), ('GASOLINE', 'VOLKSWAGEN', 'TIGUAN', 'N/A', 22, 35739, 2017), ('GASOLINE', 'INFINITI', 'QX70', 'N/A', 19, 10168, 2017), ('ETHANOL/GAS', 'CHEVROLET', 'EQUINOX', 'N/A', 24, 219482, 2017), ('GASOLINE', 'PORSCHE', 'MACAN', 'N/A', 22, 18680, 2017), ('GASOLINE', 'JAGUAR', 'F-PACE', 'N/A', 20, 18344, 2017), ('GASOLINE', 'SUBARU', 'CROSSTREK', 'N/A', 28, 70833, 2017

### Segment 9 - Small Car

In [271]:
conn.rollback()

smallcar = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 8 /* segment_ID 9 = Small Car */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(smallcar)
print_smallcar = cur.fetchall()
print(print_smallcar)

[('FIAT', '500X', 'N/A', 26, 6881, 2017), ('LINCOLN', 'MKC', 'N/A', 22, 20689, 2017), ('ACURA', 'RDX', 'N/A', 23, 38500, 2017), ('CHEVROLET', 'EQUINOX', 'N/A', 22, 219482, 2017), ('CADILLAC', 'XT5', 'N/A', 22, 47089, 2017), ('INFINITI', 'QX60', 'N/A', 22, 30338, 2017), ('ACURA', 'MDX', 'N/A', 23, 38346, 2017), ('AUDI', 'Q5', 'N/A', 22, 39592, 2017), ('FORD', 'EDGE', 'N/A', 21, 111815, 2017), ('KIA', 'SORENTO', 'N/A', 22, 72713, 2017), ('NISSAN', 'PATHFINDER', 'N/A', 23, 93520, 2017), ('NISSAN', 'ROGUE', 'N/A', 28, 354974, 2017), ('KIA', 'SPORTAGE', 'N/A', 23, 53949, 2017), ('BUICK', 'ENCORE', 'N/A', 28, 70277, 2017), ('LINCOLN', 'MKX', 'N/A', 20, 28021, 2017), ('JEEP', 'CHEROKEE', 'N/A', 24, 142643, 2017), ('MITSUBISHI', 'OUTLANDER', 'N/A', 25, 33558, 2017), ('TOYOTA', 'RAV4', 'N/A', 25, 281866, 2017), ('BUICK', 'ENVISION', 'N/A', 24, 31185, 2017), ('SUBARU', 'FORESTER', 'N/A', 26, 145166, 2017), ('TOYOTA', 'HIGHLANDER', 'N/A', 23, 163247, 2017), ('DODGE', 'JOURNEY', 'N/A', 20, 120400,

### Segment 10 - Standard SUV

In [278]:
conn.rollback()

standardSUV = """SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 10 /* segment_ID 10 = Standard SUV */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year;"""

cur.execute(standardSUV)
print_standardSUV = cur.fetchall()
print(print_standardSUV)

[('GASOLINE', 'INFINITI', 'QX80', 'N/A', 16, 18743, 2017), ('GASOLINE', 'BENTLEY', 'BENTAYGA', 'N/A', 15, 732, 2017), ('GASOLINE', 'MASERATI', 'LEVANTE', 'N/A', 16, 4358, 2017), ('GASOLINE', 'PORSCHE', 'CAYENNE', 'N/A', 20, 12740, 2017), ('GASOLINE', 'BUICK', 'ENCLAVE', 'N/A', 18, 33030, 2017), ('GASOLINE', 'LINCOLN', 'NAVIGATOR', 'N/A', 17, 8705, 2017), ('GASOLINE', 'TOYOTA', 'SEQUOIA', 'N/A', 15, 12030, 2017), ('GASOLINE', 'CHEVROLET', 'TRAVERSE', 'N/A', 18, 102732, 2017), ('GASOLINE', 'TOYOTA', '4RUNNER', 'N/A', 18, 110796, 2017), ('GASOLINE', 'VOLKSWAGEN', 'TOUAREG', 'N/A', 19, 2807, 2017), ('GASOLINE', 'FORD', 'FLEX', 'N/A', 18, 21287, 2017), ('GASOLINE', 'FORD', 'EXPEDITION', 'N/A', 18, 54047, 2017), ('GASOLINE', 'GMC', 'ACADIA', 'N/A', 22, 101892, 2017), ('GASOLINE', 'FORD', 'EXPLORER', 'N/A', 20, 219695, 2017), ('GASOLINE', 'NISSAN', 'ARMADA', 'N/A', 16, 25519, 2017), ('GASOLINE', 'AUDI', 'Q7', 'N/A', 22, 26418, 2017), ('GASOLINE', 'CADILLAC', 'ESCALADE', 'N/A', 17, 26268, 2017

### 4. Highest and lowest fuel economy within Top 10 sales segment 

### Segment 1: Pickup

In [30]:
conn.rollback()

pickup1 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 1 /* segment_ID 1 = pickup */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 1 /* segment_ID 1 = pickup */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(pickup1)
print_pickup1 = cur.fetchall()
print(print_pickup1)

[('GASOLINE', 'INFINITI', 'QX50', 'N/A', 20, 4318, 2017), ('GASOLINE', 'KIA', 'NIRO', 'N/A', 49, 4889, 2017)]


In [31]:
conn.rollback()

pickuphigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CHEVROLET'
AND mo.model LIKE 'COLORADO'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(pickuphigh)
print_pickuphigh = cur.fetchall()
print(print_pickuphigh)

[(3412, 2013), (8003, 2014), (76521, 2015), (22620, 2016), (22358, 2017)]


In [32]:
conn.rollback()

pickuplow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'TOYOTA'
AND mo.model LIKE 'TUNDRA'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(pickuplow)
print_pickuplow = cur.fetchall()
print(print_pickuplow)

[(112732, 2013), (118493, 2014), (107730, 2015), (25535, 2016), (23220, 2017)]


## Segment 2: Station Wagon

In [33]:
conn.rollback()

stwagon2 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 2 /* segment_ID 2 = station wagon */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 2 /* segment_ID 2 = station wagon */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(stwagon2)
print_stwagon2 = cur.fetchall()
print(print_stwagon2)

[('GASOLINE', 'CHRYSLER', 'PACIFICA', 'N/A', 22, 25052, 2017), ('GASOLINE', 'KIA', 'SEDONA', 'N/A', 20, 6418, 2017)]


In [35]:
conn.rollback()

stwagonhigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CHRYSLER'
AND mo.model LIKE 'PACIFICA'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(stwagonhigh)
print_stwagonhigh = cur.fetchall()
print(print_stwagonhigh)

[(140, 2016), (25052, 2017)]


In [37]:
conn.rollback()

stwagonlow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'KIA'
AND mo.model LIKE 'SEDONA'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(stwagonlow)
print_stwagonlow = cur.fetchall()
print(print_stwagonlow)

[(7079, 2013), (14567, 2014), (34439, 2015), (10231, 2016), (6418, 2017)]


## Segment 3: Special Purpose

In [38]:
conn.rollback()

sppurpose3 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 3 /* segment_ID 3 = special purpose */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 3 /* segment_ID 3 = special purpose */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(sppurpose3)
print_sppurpose3 = cur.fetchall()
print(print_sppurpose3)

[('GASOLINE', 'CHEVROLET', 'MALIBU', 'N/A', 34, 35005, 2017), ('GASOLINE', 'FORD', 'FUSION', 'N/A', 24, 50786, 2017)]


In [40]:
conn.rollback()

sppurposehigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CHEVROLET'
AND mo.model LIKE 'MALIBU'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(sppurposehigh)
print_sppurposehigh = cur.fetchall()
print(print_sppurposehigh)

[(200594, 2013), (188519, 2014), (182699, 2015), (58222, 2016), (35005, 2017)]


In [41]:
conn.rollback()

sppurposelow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'FORD'
AND mo.model LIKE 'FUSION'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(sppurposelow)
print_sppurposelow = cur.fetchall()
print(print_sppurposelow)

[(295280, 2013), (306860, 2014), (274594, 2015), (74994, 2016), (50786, 2017)]


## Segment 4: Van

In [44]:
##NOTHING IN IT

conn.rollback()

van4 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 4 /* segment_ID 4 = station wagon */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 4 /* segment_ID 4 = van */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(van4)
print_van4 = cur.fetchall()
print(print_van4)

[]


## Segment 5: Large Car

In [47]:
conn.rollback()

lcar5 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 5 /* segment_ID 5 = large car */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 5 /* segment_ID 5 = large car */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(lcar5)
print_lcar5 = cur.fetchall()
print(print_lcar5)

[('GASOLINE', 'DODGE', 'CHARGER', 'N/A', 20, 22319, 2017), ('GASOLINE', 'HYUNDAI', 'SONATA', 'N/A', 29, 37869, 2017)]


In [48]:
conn.rollback()

lcarhigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'DODGE'
AND mo.model LIKE 'CHARGER'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(lcarhigh)
print_lcarhigh = cur.fetchall()
print(print_lcarhigh)

[(98336, 2013), (94099, 2014), (86058, 2015), (26345, 2016), (22319, 2017)]


In [50]:
conn.rollback()

lcarlow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'HYUNDAI'
AND mo.model LIKE 'SONATA'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(lcarlow)
print_lcarlow = cur.fetchall()
print(print_lcarlow)

[(203648, 2013), (216936, 2014), (190483, 2015), (61457, 2016), (37869, 2017)]


## Segment 6: Minivan

In [55]:
conn.rollback()

minivan6 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
JOIN auto.segment seg on car.segment_ID = seg.segid
WHERE car.segment_ID = 6 /* segment_ID 6 = minivan */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
JOIN auto.segment seg on car.segment_ID = seg.segid
WHERE car.segment_ID = 6 /* segment_ID 6 = minivan */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(minivan6)
print_minivan6 = cur.fetchall()
print(print_minivan6)

[('GASOLINE', 'CADILLAC', 'ESCALADE', 'N/A', 17, 7505, 2017), ('GASOLINE', 'TOYOTA', 'HIGHLANDER', 'N/A', 22, 46642, 2017)]


In [56]:
conn.rollback()

minivanhigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CADILLAC'
AND mo.model LIKE 'ESCALADE'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(minivanhigh)
print_minivanhigh = cur.fetchall()
print(print_minivanhigh)

[(31264, 2015), (5021, 2016), (8107, 2016), (7505, 2017)]


In [57]:
conn.rollback()

minivanlow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'TOYOTA'
AND mo.model LIKE 'HIGHLANDER'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(minivanlow)
print_minivanlow = cur.fetchall()
print(print_minivanlow)

[(127572, 2013), (146127, 2014), (142815, 2015), (38673, 2016), (46642, 2017)]


## Segment 7: Midsize Car

In [58]:
conn.rollback()

midszcr7 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 7 /* segment_ID 7 = midsize car */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 7 /* segment_ID 7 = midsize car */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(midszcr7)
print_midszcr7 = cur.fetchall()
print(print_midszcr7)

[('GASOLINE', 'CHEVROLET', 'COLORADO', 'N/A', 15, 22358, 2017), ('GASOLINE', 'NISSAN', 'NV200', 'N/A', 25, 4495, 2017)]


In [69]:
conn.rollback()

midszcrhigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CHEVROLET'
AND mo.model LIKE 'COLORADO'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(midszcrhigh)
print_midszcrhigh = cur.fetchall()
print(print_midszcrhigh)

[(3412, 2013), (8003, 2014), (76521, 2015), (22620, 2016), (22358, 2017)]


In [71]:
conn.rollback()

midszcrlow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'NISSAN'
AND mo.model LIKE 'NV200'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(midszcrlow)
print_midszcrlow = cur.fetchall()
print(print_midszcrlow)

[(4619, 2013), (13385, 2014), (15536, 2015), (5352, 2016), (4495, 2017)]


## Segment 8: Small SUV

In [62]:
conn.rollback()

smsuv8 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 8 /* segment_ID 8 = small suv */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 8 /* segment_ID 8 = small suv */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(smsuv8)
print_smsuv8 = cur.fetchall()
print(print_smsuv8)

[('GASOLINE', 'HONDA', 'CR-V', 'N/A', 29, 94057, 2017), ('GASOLINE', 'JEEP', 'WRANGLER', 'N/A', 18, 41311, 2017)]


In [63]:
conn.rollback()

smsuvhigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'HONDA'
AND mo.model LIKE 'CR-V'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(smsuvhigh)
print_smsuvhigh = cur.fetchall()
print(print_smsuvhigh)

[(303904, 2013), (335019, 2014), (314462, 2015), (71188, 2016), (94057, 2017)]


In [65]:
conn.rollback()

smsuvlow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'JEEP'
AND mo.model LIKE 'WRANGLER'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(smsuvlow)
print_smsuvlow = cur.fetchall()
print(print_smsuvlow)

[(155502, 2013), (175328, 2014), (187111, 2015), (41922, 2016), (41311, 2017)]


## Segment 9: Small Car

In [66]:
conn.rollback()

smcar9 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 9 /* segment_ID 9 = small suv */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 9 /* segment_ID 9 = small suv */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(smcar9)
print_smcar9 = cur.fetchall()
print(print_smcar9)

[('DIESEL', 'CHEVROLET', 'CRUZE', 'N/A', 37, 53923, 2017), ('GASOLINE', 'CHEVROLET', 'CAMARO', 'N/A', 20, 15298, 2017)]


In [67]:
conn.rollback()

smcarhigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CHEVROLET'
AND mo.model LIKE 'CRUZE'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(smcarhigh)
print_smcarhigh = cur.fetchall()
print(print_smcarhigh)

[(248224, 2013), (273060, 2014), (209753, 2015), (37241, 2016), (53923, 2017)]


In [68]:
conn.rollback()

smcarlow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CHEVROLET'
AND mo.model LIKE 'CAMARO'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(smcarlow)
print_smcarlow = cur.fetchall()
print(print_smcarlow)

[(80567, 2013), (86297, 2014), (72136, 2015), (18581, 2016), (15298, 2017)]


## Segment 10: Standard SUV

In [73]:
conn.rollback()

stsuv10 = """(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 10 /* segment_ID 10 = small suv */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy DESC
LIMIT 1)

UNION

(SELECT * FROM (SELECT f.fuel, b.make, mo.model, ty.model_type, CAST(avg(fc.fuel_economy) AS INTEGER) AS fuel_economy, s.total_sales, fc.calendar_year  FROM auto.car car
JOIN auto.sales s ON car.car_ID = s.car_ID
JOIN auto.fuel_consumption fc ON car.car_ID = fc.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
JOIN auto.fuel f ON car.fuel_id = f.fuelid
WHERE car.segment_ID = 10 /* segment_ID 10 = small suv */
AND fc.calendar_year = s.calendar_year
AND fc.calendar_year = 2017
AND f.fuel NOT LIKE 'ELECTRICITY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICTY'
AND f.fuel NOT LIKE 'GASOLINE/ELECTRICITY'
AND f.fuel NOT LIKE 'HYDROGEN'
GROUP BY f.fuel, b.make, mo.model, ty.model_type, s.total_sales, fc.calendar_year
ORDER BY s.total_sales DESC
LIMIT 10) AS Top10 
ORDER BY fuel_economy ASC 
LIMIT 1);

"""

cur.execute(stsuv10)
print_stsuv10 = cur.fetchall()
print(print_stsuv10)

[('DIESEL', 'CHEVROLET', 'COLORADO', 'N/A', 24, 22358, 2017), ('GASOLINE', 'TOYOTA', 'TUNDRA', 'N/A', 16, 23220, 2017)]


In [76]:
conn.rollback()

stsuvhigh = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CHEVROLET'
AND mo.model LIKE 'COLORADO'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(stsuvhigh)
print_stsuvhigh = cur.fetchall()
print(print_stsuvhigh)

[(3412, 2013), (8003, 2014), (76521, 2015), (22620, 2016), (22358, 2017)]


In [77]:
conn.rollback()

stsuvlow = """SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'TOYOTA'
AND mo.model LIKE 'TUNDRA'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;"""

cur.execute(stsuvlow)
print_stsuvlow = cur.fetchall()
print(print_stsuvlow)

[(112732, 2013), (118493, 2014), (107730, 2015), (25535, 2016), (23220, 2017)]


### 5. Influence of fuel price on car with highest and lowest fuel economy in the top 10 sales per segment

In [ ]:
# For segment 9 lowest fuel economy (Chevrolet Camaro N/A)
SELECT DISTINCT total_sales, calendar_year from auto.sales s
JOIN auto.car car ON car.car_ID = s.car_ID
JOIN auto.brand b ON car.brand_id = b.brandid
JOIN auto.model mo ON car.model_id = mo.modelid
JOIN auto.type ty ON car.type_id = ty.typeid
WHERE b.make LIKE 'CHEVROLET'
AND mo.model LIKE 'CAMARO'
AND ty.model_type LIKE 'N/A'
ORDER BY calendar_year ASC;

In [ ]:
# For segment 9 highest fuel economy (Chevrolet Cruze N/A)


# Group Contribution